In [13]:
#@title Enter the construct sequences in FASTA format and hit `Runtime` -> `Run all`
batch_size = 4 # @param {"type":"integer"}
input_fasta = """
>CONSTR_000001
MTVFFVTRLVKKHDKLSKQQIEDFAEKLMTILFETYRSHWHSDCPSKGQAFRCIRINNNQ
NKDPILERACVESNVDFSHLGLPKEMTIWVDPFEVCCRYGEKNHPFTVASFKGRWEEWEL
YQQISYAVSRASSDVSSGTSCDEESCGSHHHHHH
>CONSTR_000002
MDYTKPLEHPPVKRNEEAQVHDKLNSGMVSNMEGTAGGERPSVVNGDSGKSGGVGDPREP
LGCLQEGSGCHPTTESFEKSVREDASPLPHVCCCKQDALILQRGLHHEDGSQHIGLLHPG
DRGPDHEYVLVEEAECGSHHHHHH
>CONSTR_000003
MHHHHHHENLYFQGSLEVRGQLQSALLILGEPKEGGMPMNISIMPSSLQMKTPEGCTEIQ
LPAEVRLVPSSCRGLQFVVGDGLHLRLQTQAKLGTKLISMFNQSSQTQE
>CONSTR_000004
MECPEGQLPISSENDSTPTVSTSEVTSQQEPQILVDRGSETTYESSADIAGDEGTQIPAD
EDTQTDADSSAQAAAQAPENFQEGKDMSESQDEVPDEVENGSHHHHHH
>CONSTR_000005
MSTAPSEDIWKKFELVPSPPTSPPWGLGPGAGDPAPGIGPPEPWPGGCTGDEAESRGHSK
GWGRNYASIIRRDCMWSGFSARERLERAVSDRLAPGAPRGNPPKASAAPDCTPSLEAGNP
APAAPCPLGEPKTQACSGSESPSDSENEEIDVVTVEKRQSLGIRKPVTITVRADPLDPCM
KHFHGSHHHHHH
>CONSTR_000006
MEKARHETFAAEMRQNDKIMCILENRKKRDRKNLCRAINDFQQSFQKPETRREFDLSDPL
ALKKDLPARQSDNDVRNTISGMQGSHHHHHH
>CONSTR_000007
MLMKKAYELSVLCDCEIALIIFNSANRLFQYASTDMDRVLLKYTEYSEPHESRTNTDILE
TLKRRGIGLDGPELEPDEGPEEPGEKFRRLAGEGGDPGSHHHHHH
>CONSTR_000008
MPTESASCSTARQTKQKRKSHSLSIRRTNSSEQERTGLPRDMLEGQDSKLPSSVRSTLLE
LFGQIEREFENLYIENLELRREIDTLNERLAAEGQAIDGAELSKGQLKTKASHSTSQLSQ
KLKTTYKASTSKIVSSFKTTTSRAACQLVKEYIGHRDGIWDVSVAKTQPVVLGTASADHT
ALLWSIETGKCLVKYAGHVGSVNSIKFHPSEQLALTASGDQTAHIWRYAVQLPTPQPVAD
TSISGEDEVECSDKDEPDLDGDVSSDCPTIRVPLTSLKSHQGVVIASDWLVGGKQAVTAS
WDRTANLYDVETSELVHSLTGHDQELTHCCTHPTQRLVVTSSRDTTFRLWDFRDPSIHSV
NVFQGHTDTVTSAVFTVGDNVVSGSDDRTVKVWDLKNMRSPIATIRTDSAINRINVCVGQ
KIIALPHDNRQVRLFDMSGVRLARLPRSSRQGHRRMVCCSAWSEDHPVCNLFTCGFDRQA
IGWNINIPALLQEKGSHHHHHH
>CONSTR_000009
MHHHHHHENLYFQGSPTESASCSTARQTKQKRKSHSLSIRRTNSSEQERTGLPRDMLEGQ
DSKLPSSVRSTLLELFGQIEREFENLYIENLELRREIDTLNERLAAEGQAIDGAELSKGQ
LKTKASHSTSQLSQKLKTTYKASTSKIVSSFKTTTSRAACQLVKEYIGHRDGIWDVSVAK
TQPVVLGTASADHTALLWSIETGKCLVKYAGHVGSVNSIKFHPSEQLALTASGDQTAHIW
RYAVQLPTPQPVADTSISGEDEVECSDKDEPDLDGDVSSDCPTIRVPLTSLKSHQGVVIA
SDWLVGGKQAVTASWDRTANLYDVETSELVHSLTGHDQELTHCCTHPTQRLVVTSSRDTT
FRLWDFRDPSIHSVNVFQGHTDTVTSAVFTVGDNVVSGSDDRTVKVWDLKNMRSPIATIR
TDSAINRINVCVGQKIIALPHDNRQVRLFDMSGVRLARLPRSSRQGHRRMVCCSAWSEDH
PVCNLFTCGFDRQAIGWNINIPALLQEK
>CONSTR_000010
MRDEIATTVFFVTRLVKKHDKLSKQQIEDFAEKLMTILFETYRSHWHSDCPSKGQAFRCI
RINNNQNKDPILERACVESNVDFSHLGLPKEMTIWVDPFEVCCRYGEKNHPFTVASFKGR
WEEWELYQQISYAVSRASSDVSSGTSCDEESCSKEPRVIPKVSNPKSIYQVENLKQPFQS
WLQIPRKKNVVDGRVGLLGNTYHGSQKHPKCYRPAMHRLDRILGSHHHHHH
"""

In [2]:
#@title Install dependencies and download the checkpoint; takes about 2min
%%bash

set -e

pip install peft==0.12.* RP3Net
wget -nv -nc https://ftp.ebi.ac.uk/pub/software/RP3Net/v0.1/checkpoints/rp3net_v0.1_d.ckpt

In [20]:
#@title Imports
import re
import io
import pandas as pd
import RP3Net as rp3
from tqdm.notebook import tqdm
RE_FASTA_HEADER = re.compile(r'^>([\w\-.:#*]+)') # https://www.ncbi.nlm.nih.gov/genbank/fastaformat/

In [18]:
#@title Helper functions
def iter_fasta(io):
    fasta_id, sequence = None, None
    for line in io:
        line = line.strip()
        if len(line) == 0:
            continue
        m = RE_FASTA_HEADER.match(line)
        if m:
            if fasta_id is not None:
                yield fasta_id, ''.join(sequence)
            sequence = []
            fasta_id = m.group(1)
        else:
            sequence.append(line)
    if fasta_id is not None:
        yield fasta_id, ''.join(sequence)

def parse_fasta(s):
    return {id: seq for id, seq in iter_fasta(io.StringIO(s))}

def batches():
    fasta_map = parse_fasta(input_fasta)
    fasta_keys = list(fasta_map.keys())
    r = tqdm(range(0, len(fasta_map), batch_size), desc='RP3Net')
    for i in r:
        yield {k: fasta_map[k] for k in fasta_keys[i:i + batch_size]}

In [19]:
#@title Load the model
m = rp3.load_model(rp3.RP3_DEFAULT_CONFIG, 'rp3net_v0.1_d.ckpt')


RP3Net:   0%|          | 0/2 [00:00<?, ?it/s]

CONSTR_000001,0.6903
CONSTR_000002,0.9721
CONSTR_000003,0.9271
CONSTR_000004,0.9730
CONSTR_000005,0.9240
CONSTR_000006,0.9772
CONSTR_000007,0.7405
CONSTR_000008,0.0090
CONSTR_000009,0.0099
CONSTR_000010,0.4248


In [ ]:
#@title Run the prediction
scores_map = dict()
for b in batches():
    scores_map |= m.predict(b)


In [23]:
#@title Print and save the results
df = pd.DataFrame([[id, score] for (id, score) in scores_map.items()], columns=['id', 'score'])
print(df)
df.to_csv("rp3_scores.csv", index=False)

              id     score
0  CONSTR_000001  0.690350
1  CONSTR_000002  0.972059
2  CONSTR_000003  0.927142
3  CONSTR_000004  0.972956
4  CONSTR_000005  0.924033
5  CONSTR_000006  0.977236
6  CONSTR_000007  0.740516
7  CONSTR_000008  0.009047
8  CONSTR_000009  0.009949
9  CONSTR_000010  0.424805
